# Kayıp Fonksiyonları

Bu alıştırmada, Kayıp fonksiyonlarının `LinearRegression` modeli üzerindeki etkilerini karşılaştıracaksınız.

👇 Bu zorluk için kullanmak üzere bir CSV dosyası indirelim ve onu bir DataFrame'e dönüştürelim

In [15]:
import pandas as pd

data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/loss_functions_dataset.csv")
data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
470,0.66,759.5,318.5,220.50,3.5,0.25,14.515
193,0.98,514.5,294.0,110.25,7.0,0.10,24.975
339,0.98,514.5,294.0,110.25,7.0,0.25,29.310
410,0.74,686.0,245.0,220.50,3.5,0.25,13.350
604,0.71,710.5,269.5,220.50,3.5,0.40,15.735


🎯 Göreviniz, tasarımına göre bir seranın içindeki ortalama sıcaklığı tahmin etmektir. Sıcaklık tahminleriniz, her bir bitki için iklim ihtiyaçlarına göre uygun sera tasarımını seçmenize yardımcı olacaktır.

🌿 Bitkilerin küçük sıcaklık değişimlerini kaldırabildiğini, ancak sıcaklık değişimleri arttıkça katlanarak daha duyarlı hale geldiğini biliyorsunuz.

## 1. Teori

❓ Teorik olarak, bitkileri öldürme riskini sınırlamak için modelinizi hangi Kayıp fonksiyonu üzerinde eğitirsiniz?

<details>
<summary> 🆘 Cevap </summary>
    
Teorik olarak, Ortalama Kare Hata (MSE) Kayıp fonksiyonunu kullanırsınız. Bu, aykırı tahminleri cezalandırır ve modelinizin büyük hatalar yapmasını engeller. Bu, daha küçük sıcaklık değişimleri ve bitkiler için daha düşük risk sağlayacaktır.

</details>

> CEVABINIZI BURAYA YAZIN : Hata büyüdükçe cezayı katlayarak artıran yöntem MSE'dir.

## 2. Uygulama

### 2.1 Ön İşleme

❓ Özellikleri standartlaştırın

In [16]:
from sklearn.preprocessing import StandardScaler

X = data.drop(columns=["Average Temperature"])
y = data["Average Temperature"]

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

print("Data standardized.")
print(X_scaled[:0])

Data standardized.
[]


### 2.2 Modelleme

Bu bölümde, farklı Kayıp fonksiyonları üzerinde optimize edilmiş modelleri değerlendirerek teoriyi doğrulayacaksınız.

### En Küçük Kareler (MSE) Kaybı

❓ **En Küçük Kareler Kaybı** (MSE) üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

In [17]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_validate

model = SGDRegressor(loss="squared_error", max_iter=1000, random_state=42)

cv_results = cross_validate(model, X_scaled, y, cv=10, scoring='r2')

print("Average Score (R²):", cv_results['test_score'].mean())

Average Score (R²): 0.8982272875896993


❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru ve bunu `r2` değişkeninde kaydedin
- Tüm katlarınızın °C cinsinden en büyük tek tahmin hatasını hesaplayın ve `max_error_celsius` değişkeninde kaydedin

(İpucu: `max_error` sklearn'de kabul edilen bir puanlama metriğidir)

In [18]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_validate

model = SGDRegressor(loss="squared_error", max_iter=1000, random_state=42)

cv_results = cross_validate(model, X_scaled, y, cv=10, scoring=['r2', 'max_error'])

r2 = cv_results['test_r2'].mean()
max_error_celsius = abs(cv_results['test_max_error'].min())

print("Average Score (R²):", r2)
print("Biggest Single Mistake:", max_error_celsius)

Average Score (R²): 0.8982272875896993
Biggest Single Mistake: 9.787281346962047


### Ortalama Mutlak Hata (MAE) Kaybı

Peki modelimizi MAE üzerinde optimize edersek ne olur?

❓ **MAE** Kaybı üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

<details>
<summary>💡 İpuçları</summary>

- MAE kaybı `SGDRegressor`'da doğrudan belirtilemez. Doğru parametreleri ayarlayarak tasarlanması gerekir

</details>

In [23]:
model_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0, max_iter=1000, random_state=42)

cv_results_mae = cross_validate(model_mae, X_scaled, y, cv=10, scoring=['r2', 'max_error'])

r2_mae = cv_results_mae['test_r2'].mean()
max_error_mae = abs(cv_results_mae['test_max_error'].min())

print("Model MAE R2 Score:", cv_results_mae['test_r2'].mean())
print("MAE Model Biggest Mistake:", abs(cv_results_mae['test_max_error'].min()))

Model MAE R2 Score: 0.8763275370982454
MAE Model Biggest Mistake: 11.210013223184902


❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru, bunu `r2_mae`'de saklayın
- Tüm katlarınızın en büyük tek tahmin hatasını, bunu `max_error_mae`'de saklayın

In [20]:
r2_mae = cv_results_mae['test_r2'].mean()
max_error_mae_celsius = abs(cv_results_mae['test_max_error'].min())

print ("Model MAE R2 Score:", r2_mae)
print ("MAE Model Biggest Mistake:", max_error_mae_celsius)

Model MAE R2 Score: 0.8763275370982454
MAE Model Biggest Mistake: 11.210013223184902


## 3. Sonuç

❓ Değerlendirdiğiniz modellerden hangisi göreviniz için en uygun görünüyor?

<details>
<summary> 🆘Cevap </summary>
    
İki model arasında ortalama çapraz doğrulanmış r2 skorları yaklaşık olarak benzer olmasına rağmen, MAE üzerinde optimize edilen modelin zaman zaman daha büyük hatalar yapma şansı daha fazladır, bu da bitkileri öldürme riskini artırır!
    
</details>

> CEVABINIZI BURAYA YAZIN : MSE modeli kesinlikle daha uygundur çünkü mae'nin yaptığı en büyük hata 11 iken mse'nin yaptığı en büyük hata 9.78'di. 

# 🏁 Kodunuzu kontrol edin ve notebook'unuzu gönderin

In [24]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/berkayturhan/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/berkayturhan/S16D4-S-loss-functions/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.16s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

